1. IMPORT AND INSTALL DEPENDENCIES

In [1]:
%pip install tensorflow==2.13.0 tensorflow-gpu opencv-python sklearn matplotlib mediapipe

  Using cached tensorflow-2.13.0-cp310-cp310-win_amd64.whl (1.9 kB)
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl (38.1 MB)
  Using cached sklearn-0.0.post7.tar.gz (3.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 276.5/276.5 MB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 4.3/4.3 MB 1.3 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     -------------------------------------- 440.8/440.8 KB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 1.5 MB/s eta 0:00:00
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl 

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [18 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Pradip Kumar Singh\AppData\Local\Temp\pip-install-mmr76nn9\tensorflow-gpu_e826e80f1ce348b4922be8de1aafbd37\setup.py", line 37, in <module>
          raise Exception(TF_REMOVAL_WARNING)
      Exception:
      
      The "tensorflow-gpu" package has been removed!
      
      Please install "tensorflow" instead.
      
      Other than the name, the two packages have been identical
      since TensorFlow 2.1, or roughly since Sep 2019. For more
      information, see: pypi.org/project/tensorflow-gpu
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error:

In [3]:
import cv2
import numpy as np
import mediapipe as mp
import os
import time
from matplotlib import pyplot as plt

2. Marking Keypoints Using Mediapipe Holistic

In [6]:
#Open Camera
cap = cv2.VideoCapture(0)

while cap.isOpened():
    #Read frame
    ret, frame = cap.read()
    #Show on screen
    cv2.imshow('OpenCV Window', frame)
    if cv2.waitKey(10) & 0xFF== ord('q'):
        break
cap.release()
cv2.destroyAllWindows()